# Previsão de Doenças Cardiovasculares

O objetivo é prever a variável binária : ** Heart Disease Status ** .Sendo este, um problema de classificação, ideal para treinar modelos como Random Forest, Decision Tree ...

## Dataset

O dataset elegido para este estudo foi **"Heart Disease Data for Health Research**, que contém características (features) que são cruciais no mundo real para o estudo de doeças cardiovascular, como:

> - Feature 1 - explicar
>
> - Feature 2 - explicar
>
> - Feature 3 - explicar
>

Podem ainda, ser considerados os seguintes atributos de risco: `feature 1, feature 2 `

### Necessário

> **Dataset** nome.csv
>
> Obtido em :  `https://www.kaggle.com/datasets/oktayrdeki/heart-disease`


**Questões adicionais**

-Quais variáveis mais influenciam o risco? (feature importance)

-Modelos mais simples funcionam tão bem quanto modelos complexos?

-Podemos otimizar o recall sem perder muita precisão?

-Existe algum grupo com maior risco (ex.: idade, sexo)?
        

In [3]:
!pip install mlxtend
!pip install catboost
!pip install xgboost
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from mlxtend.frequent_patterns import apriori, association_rules

import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

import joblib

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['figure.dpi'] = 120


In [14]:
# import dataset
dataset_path = "./heart_disease.csv"
df = pd.read_csv(dataset_path)

display(df.head())

# Debugg Extra
#print(df.columns)
#print(df.shape)
#print(df.info()) 

,Age,Gender,Blood Pressure,Cholesterol Level,Exercise Habits,Smoking,Family Heart Disease,Diabetes,BMI,High Blood Pressure,...,High LDL Cholesterol,Alcohol Consumption,Stress Level,Sleep Hours,Sugar Consumption,Triglyceride Level,Fasting Blood Sugar,CRP Level,Homocysteine Level,Heart Disease Status
0,56.0,Male,153.0,155.0,High,Yes,Yes,No,24.991591,Yes,...,No,High,Medium,7.633228,Medium,342.0,NaN,12.969246,12.387250,No
1,69.0,Female,146.0,286.0,High,No,Yes,Yes,25.221799,No,...,No,Medium,High,8.744034,Medium,133.0,157.0,9.355389,19.298875,No
2,46.0,Male,126.0,216.0,Low,No,No,No,29.855447,No,...,Yes,Low,Low,4.440440,Low,393.0,92.0,12.709873,11.230926,No
3,32.0,Female,122.0,293.0,High,Yes,Yes,No,24.130477,Yes,...,Yes,Low,High,5.249405,High,293.0,94.0,12.509046,5.961958,No
4,60.0,Male,166.0,242.0,Low,Yes,Yes,Yes,20.486289,Yes,...,No,Low,High,7.030971,High,263.0,154.0,10.381259,8.153887,No


## Análise Exploratória de Dados
De forma a entender a estrutura, qualidade e as relações subjacentes do dataset, e tendo em conta que o DataFrame é tanto numérico padronizado e categórico, a análise exploratória deve ser focada em duas áreas principais, a Visão Geral e Análise Bivariada.


In [16]:
numeric_cols = [
    "age", "Blood Pressure", "Cholesterol Level", "BMI", "Sleep Hours", 
    "Triglyceride Level", "Fasting Blood Sugar", "CRP Level", "Homocysteine Level"
]

print("Informações do DataFrame:\n")
display(df.info())

print("\nValores Ausentes por Coluna:\n")
display(df.isnull().sum().sort_values(ascending=False).head())

status_counts = df["Heart Disease Status"].value_counts()
display(status_counts)

Informações do DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   9971 non-null   float64
 1   Gender                9981 non-null   object 
 2   Blood Pressure        9981 non-null   float64
 3   Cholesterol Level     9970 non-null   float64
 4   Exercise Habits       9975 non-null   object 
 5   Smoking               9975 non-null   object 
 6   Family Heart Disease  9979 non-null   object 
 7   Diabetes              9970 non-null   object 
 8   BMI                   9978 non-null   float64
 9   High Blood Pressure   9974 non-null   object 
 10  Low HDL Cholesterol   9975 non-null   object 
 11  High LDL Cholesterol  9974 non-null   object 
 12  Alcohol Consumption   7414 non-null   object 
 13  Stress Level          9978 non-null   object 
 14  Sleep Hours           9975 non-null   float6

None


Valores Ausentes por Coluna:



Alcohol Consumption    2586
Cholesterol Level        30
Sugar Consumption        30
Diabetes                 30
Age                      29
dtype: int64

Heart Disease Status
No     8000
Yes    2000
Name: count, dtype: int64

### Análise Gráfica do Dataset

Para confirmar a validade e o insight das Regras de Associação iremos apresentar um Count Plot do **Heart Disease Status** de forma a revelar o balanceamento da variável alvo, prevenindo modelos enviesados.


In [28]:
df['Age'] = pd.cut(
    df['Age'],
    bins=[18,30,45,60, np.inf],
    labels = ['18-30_Jovem', '31-45_Adulto_Jovem', '46-60_Adulto', '60+_Senior'],
    right=False
)

# Visualização da Distribuição
plt.figure(figsize=(6, 4))
sns.countplot(x='Heart Disease Status', data=df)
plt.title('Balanceamento das Classes (Heart Disease status)')
plt.show()

# Box Plot para Cholesterol Level vs Heart Disease Status
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.boxplot(x='Heart Disease Status', y='Cholesterol Level', data=df)
plt.title('Cholesterol Level vs Heart Disease Status')


TypeError: '<' not supported between instances of 'float' and 'str'

## Limpeza do Dataset

Antes de procedemosao treinamento dos modelos de AI, iremos verificar se o dataset está pronto para os processos de ML.
Verificamos se existem colunas vazias ou com valores nulos, como a rotulação de variáveis strings para que estas passem a valores binários de forma a que os modelos ML consigam entender estes valores.

#### Codificação Categórica

De forma a que o modelo ML consiga entender os valores é necessario passar vaores string para valores binários de forma a que o modelo ML consiga entender os valores do dataset  entenda o valor das nossas variáveis vamos aplicar um Label Encoding - para variáveis binárias, para transformar strings em binário, 0 e 1 

Como o Heart Disease Status é o target, codificamos de forma binária para facilitar a modelagem.

Para variáveis > 2 categorias, usamos o One-Hot Enconding



In [50]:
#dataset= dataset.drop_duplicates(inplace = True)

# Verificação rápida se existem colunas com valores Na
missing_values = dataset.isna().sum()
print(missing_values)

C:\Users\vasco\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\vasco\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
C:\Users\vasco\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\vasco\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


AttributeError: 'NoneType' object has no attribute 'isna'

In [26]:
binary_cols = ['Gender', 'Smoking', 'Family Heart Disease', 'Diabetes', 
               'High Blood Pressure', 'High LDL Cholesterol', 'Heart Disease Status']

multi_cat_cols = ['Exercise Habits', 'Alcohol Consumption', 'Stress Level', 'Sugar Consumption']

# Label Enconding
for col in binary_cols:
    dataset[col] = dataset[col].map({
        'No':0,
        'Yes':1,
        'Male':1,
        'Female':0
    })

#One-Hot Encoding
dataset = pd.get_dummies(dataset, cols=multi_cat_cols, drop_first=True)

display(dataset)

TypeError: 'NoneType' object is not subscriptable

## Normalização

Para uniformizar os dados, dentro de um intervalo devemos normalizar os daos usando o ** Nome do Metodo **

In [32]:
numeric_columns = dataset.select_dtypes(include=['float64', 'int64']).columns
dataset_scaled = dataset.copy()
dataset_scaled[numeric_columns] = StandardScaler().fit_transform(dataset[numeric_columns])

print(dataset_scaled)

AttributeError: 'NoneType' object has no attribute 'select_dtypes'